# Recomendador de Productos Solares con Agente Interactivo (Nivel Básico)

Este proyecto implementa un **sistema de recomendación interactivo** para productos de energía solar, usando:

- **Python, Pandas y Scikit-Learn**
- **TF-IDF + Cosine Similarity**
- **Agente conversacional con inputs del usuario**

### Funcionalidades:
✔ Filtrar por categoría
✔ Ajustar por presupuesto máximo
✔ Seleccionar palabras clave deseadas
✔ Recomendaciones inteligentes usando similitud de contenido

---

In [ ]:
# Instalación de dependencias necesarias
!pip install nltk
import nltk
nltk.download('stopwords')

In [ ]:
# Importación de librerías
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ruta del archivo Excel del dataset
file_path = '/content/drive/MyDrive/IA/proyecto/productos_energia_solar.xlsx'

# Leer dataset
df = pd.read_excel(file_path)
df.head()

In [ ]:
# Crear columna combinada para análisis semántico
df['contenido'] = df['descripcion'] + ' ' + df['palabras_clave'] + ' ' + df['categoría']

In [ ]:
# Crear matriz TF-IDF con stopwords en español
spanish_stopwords = stopwords.words('spanish')
vectorizer = TfidfVectorizer(stop_words=spanish_stopwords)
tfidf_matrix = vectorizer.fit_transform(df['contenido'])

In [ ]:
# Crear matriz de similitud
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Agente interactivo para recomendar productos solares
def agente_recomendador_solar():
    print("🔆 ¡Hola! Soy tu asistente para encontrar productos solares ideales.")

    while True:
        categoria = input("\n¿Qué tipo de producto buscas? (o escribe 'salir' para terminar): ").strip()
        if categoria.lower() == 'salir':
            print("¡Gracias por usar el asistente solar! ☀️")
            break

        if categoria not in df['categoría'].unique():
            print("❌ Categoría no encontrada. Intenta con otra.")
            continue

        presupuesto = input("¿Cuál es tu presupuesto máximo (en COP)? (o presiona Enter para omitir): ").strip()
        presupuesto = int(presupuesto) if presupuesto.isdigit() else None

        palabras = input("¿Qué características deseas? (Ej.: litio, inversor, techo, separados por coma): ").strip().lower()
        texto_usuario = categoria + ' ' + palabras

        resultados = df[df['categoría'] == categoria]
        if presupuesto:
            resultados = resultados[resultados['precio'] <= presupuesto]

        if resultados.empty:
            print("⚠️ No encontramos productos con esos filtros. Intenta ajustarlos.")
            continue

        vector_usuario = vectorizer.transform([texto_usuario])
        similitudes = cosine_similarity(vector_usuario, vectorizer.transform(resultados['contenido'])).flatten()
        resultados['similitud'] = similitudes
        recomendaciones = resultados.sort_values(by='similitud', ascending=False).head(5)

        print("\n✅ Productos recomendados:")
        for i, row in recomendaciones.iterrows():
            print(f"\n🔋 Producto: {row['nombre']}")
            print(f"💰 Precio: {row['precio']} COP")
            print(f"📦 Categoría: {row['categoría']}")
            print(f"📝 Descripción: {row['descripcion']}")

        continuar = input("\n¿Quieres buscar otro producto? (sí/no): ").strip().lower()
        if continuar != 'sí':
            print("¡Hasta pronto! 🌞")
            break

# Ejecutar agente
agente_recomendador_solar()